In [2]:
!pip install redis

    100% |████████████████████████████████| 61kB 2.6MB/s ta 0:00:01
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from redis import StrictRedis
from urllib.parse import urlencode
import requests
import json

In [8]:
urlencode({'abc': ('xyz', 'jkl')})

'abc=%28%27xyz%27%2C+%27jkl%27%29'

In [20]:
r = StrictRedis(host='52.41.187.171')

In [21]:
class WikiApi:

    def presets(presets):
        presets = {**presets}
        def request(request_func):
            def execute(self, *args, **params):
                params['query']=urlencode({**presets, **params})
                return request_func(self, *args, **params)
            return execute
        return request
    api_defaults={'action':'query', 'format':'json'}
    
    
    
    def __init__(self, redis_host, api='http://en.wikipedia.org/w/api.php?',
                 api_defaults={'action':'query', 'format':'json'}, 
                 subcat_key_format='subcats_{}', page_key_format='pages_{}',
                 **redis_params):
        '''
        Initialize with the address of the API, a redis connection,
            redis key format strings for subcategory and page lists,
            and a variable to store queries.
        '''
        self._api = api
        self._api_defaults = api_defaults
        self._redis = StrictRedis(redis_host, **redis_params)
        self._subcat_key_fmt = subcat_key_format
        self._page_key_fmt = page_key_format

        
        
    @presets({'list':'categorymembers', 'cmtype':'page', 'cmlimit':'max'})
    def get_pages(self, category, subcat_depth=0, query=None, **params):
        '''
        Submits request to Wikipedia for category information
        -- Parameters --
        category: Name of the to category to get pages from
        params: Any additional key-value pairs for the API request
        Returns API response in JSON format
        '''
        return self._query_category(category, self._page_key_format,**params)
        
        
    
    @presets({'list':'categorymembers', 'cmtype':'subcat', 'cmlimit':'max'})
    def get_subcats(self, category, subcat_depth=0, query=None, **params):
        '''
        Submits request to Wikipedia for category information
        -- Parameters --
        category: Name of the to category to get page list from
        subcat_depth: Max # of levels of subcategories to get
        params: Any additional key-value pairs for the API request
        Returns API response in JSON format
        '''
        data = self._query_category(category, self._subcat_key_format, **params)
        if subcat_depth > 0:
            for subcat in data:
                data += self.subcategories(subcat['title'], subcat_depth - 1, **params)
        return data
        

        
    @presets({'prop':'revision', 'rvprop':'content', 'limit':'max'})
    def get_content(self, ref_type='titles', *refs, query=None, **params):
        '''
        Submits request to Wikipedia for content of page(s)
        -- Parameters --
        ref_type: Type of page reference - either 'titles' or 'pageids'
        refs: Page title(s) or pageid(s), as strings.
        params: Any additional key-value pairs for the API request
        Returns API response in JSON format
        '''
        if ref_type != 'titles' and ref_type != 'pageids':
            raise ValueError("ref_type must be 'titles' or 'pageids'")
            
        self._query = '&'.join([
            self._query,
            tasty_params(**{ref_type: refs}, **params)
        ])
        return requests.get(self._query).json()
    
    
        
    def _query_category(self, query, key_format, **params):
        '''
        Submits request to Wikipedia and incrementally saves results in redis
        -- Parameters --
        category: Name of the to category to get page list from
        key_format: Redis key format string to format with category
        params: Any additional key-value pairs for the API request
        Returns dictionary of results.
        '''
        key = key_format.format(category)
        resp = requests.get(self._query).json()
        data = resp['query']['categorymembers']
        self._redis.set(key, data)
        while 'continue' in resp.keys():
            to_append = '&cmcontinue=' + resp['continue']['cmcontinue']
            resp = requests.get(self._query + to_append).json()
            data += resp['query']['categorymembers']
            self._redis.set(key, json.dumps(data))
        return data